## Plan-and-Execute란 무엇인가?

"plan-and-execute"는 다음과 같은 특징을 갖는 접근 방식입니다.

- **장기 계획 수립**: 복잡한 작업을 수행하기 전에 큰 그림을 그리는 장기 계획을 수립합니다.
- **단계별 실행 및 재계획**: 세운 계획을 단계별로 실행하고, 각 단계가 완료될 때마다 계획이 여전히 유효한지 검토한 뒤 수정할 수 있습니다.
  
이 방식은 [Plan-and-Solve 논문](https://arxiv.org/abs/2305.04091)과 [Baby-AGI 프로젝트](https://github.com/yoheinakajima/babyagi)에서 영감을 받았습니다. 전통적인 [ReAct 스타일](https://arxiv.org/abs/2210.03629)의 에이전트는 한 번에 한 단계씩 생각하는 반면, "plan-and-execute"는 명시적이고 장기적인 계획을 강조합니다.

**장점**:
1. **명시적인 장기 계획**: 강력한 LLM조차도 한 번에 장기 계획을 처리하는 데 어려움을 겪을 수 있습니다. 명시적으로 장기 계획을 수립함으로써, 보다 안정적인 진행이 가능합니다.
2. **효율적인 모델 사용**: 계획 단계에서는 더 큰/강력한 모델을 사용하고, 실행 단계에서는 상대적으로 작은/약한 모델을 사용함으로써 자원 소비를 최적화할 수 있습니다.


by. 테디노트

In [ ]:
from langchain_teddynote.models import get_model_name, LLMs

# 모델명 정의
MODEL_NAME = get_model_name(LLMs.GPT4o)
print(MODEL_NAME)

gpt-4o


## 도구 정의

사용할 도구를 먼저 정의합니다. 이 간단한 예제에서는 `Tavily`를 통해 제공되는 내장 검색 도구를 사용할 것입니다. 그러나 직접 도구를 만드는 것도 매우 쉽습니다. 

자세한 내용은 [도구(Tools)](https://wikidocs.net/262582) 문서를 참조하십시오.

In [ ]:
from langchain_teddynote.tools import TavilySearch

# Tavily 검색 도구 초기화
tools = [TavilySearch(max_results=3)]

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from typing import List
import glob
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# 캐시 디렉토리 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

# DB 저장 경로 설정
PERSIST_DIRECTORY = os.path.join('C:/Users/user/Documents/langchain-kr/어진_RFP', 'db')
PDF_DIRECTORY = 'C:/Users/user/Documents/langchain-kr/어진_RFP'

def get_embeddings():
    """HuggingFace 임베딩 모델 설정"""
    model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cpu"}  # CPU 사용
    encode_kwargs = {"normalize_embeddings": True}
    
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder="./cache/"
    )

def load_all_pdfs() -> List[str]:
    """폴더 내의 모든 PDF 파일 경로를 반환"""
    pdf_pattern = os.path.join(PDF_DIRECTORY, '*.pdf')
    return glob.glob(pdf_pattern)

def load_and_persist_pdfs():
    """모든 PDF를 로드하고 Chroma DB에 저장"""
    pdf_files = load_all_pdfs()
    print(f"Found {len(pdf_files)} PDF files")
    
    all_documents = []
    for pdf_path in pdf_files:
        print(f"Loading {os.path.basename(pdf_path)}...")
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        all_documents.extend(documents)
    
    # 텍스트 분할 (청크 크기 조정)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # 더 작은 청크 크기로 조정
        chunk_overlap=50,
        length_function=len
    )
    splits = text_splitter.split_documents(all_documents)
    print(f"Created {len(splits)} text chunks")
    
    # HuggingFace 임베딩 사용
    embeddings = get_embeddings()
    
    # Chroma DB에 저장
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
    return vectorstore

def load_existing_db():
    """기존 DB 로드"""
    embeddings = get_embeddings()
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    return vectorstore

def get_vectorstore():
    """DB가 없으면 생성, 있으면 로드"""
    if not os.path.exists(PERSIST_DIRECTORY):
        print("Creating new Chroma DB...")
        return load_and_persist_pdfs()
    else:
        print("Loading existing Chroma DB...")
        return load_existing_db()

# Chroma DB 초기화
try:
    vectorstore = get_vectorstore()
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}
    )
    print("Successfully initialized Chroma DB and retriever")
except Exception as e:
    print(f"Error initializing Chroma DB: {str(e)}")

Creating new Chroma DB...
Found 13 PDF files
Loading 01_2023미융파_RFP_메타초음파_최종.pdf...
Loading 02_2023미융파_RFP_탄소중립_최종.pdf...
Loading 03_2023미융파_RFP_이종집적_최종.pdf...
Loading 04_2023미융파_RFP_스마트건설_최종.pdf...
Loading 05_2023미융파_RFP_광통신_최종.pdf...
Loading 2023-과학기술인문사회융합연구-02.pdf...
Loading 2023-브릿지융합연구-03.pdf...
Loading 2023-브릿지융합연구-04.pdf...
Loading 2023-브릿지융합연구-05.pdf...
Loading 2023-브릿지융합연구-06.pdf...
Loading 2023-브릿지융합연구-07.pdf...
Loading 2023년도 제2차 범부처전주기의료기기연구개발사업 신규지원 대상과제 과제제안요청서(RFP).pdf...
Loading 붙임 1. 2022년도 제2차 범부처전주기의료기기연구개발사업 신규지원 대상과제 제안요청서(RFP).pdf...
Created 325 text chunks


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Successfully initialized Chroma DB and retriever


In [ ]:
import os
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.models import get_model_name, LLMs
from langchain_teddynote.tools import TavilySearch
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from typing import List
import glob
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# API 키 로드
load_dotenv()

# LangSmith 설정
logging.langsmith("RFP-Assistant")

# 모델명 정의
MODEL_NAME = get_model_name(LLMs.GPT4o)

# 캐시 디렉토리 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

# DB 저장 경로 설정
PERSIST_DIRECTORY = os.path.join('C:/Users/user/Documents/langchain-kr/어진_RFP', 'db')
PDF_DIRECTORY = 'C:/Users/user/Documents/langchain-kr/어진_RFP'

def get_embeddings():
    """HuggingFace 임베딩 모델 설정"""
    model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {"normalize_embeddings": True}
    
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder="./cache/"
    )

def load_all_pdfs() -> List[str]:
    """폴더 내의 모든 PDF 파일 경로를 반환"""
    pdf_pattern = os.path.join(PDF_DIRECTORY, '*.pdf')
    return glob.glob(pdf_pattern)

def load_and_persist_pdfs():
    """모든 PDF를 로드하고 Chroma DB에 저장"""
    pdf_files = load_all_pdfs()
    print(f"Found {len(pdf_files)} PDF files")
    
    all_documents = []
    for pdf_path in pdf_files:
        print(f"Loading {os.path.basename(pdf_path)}...")
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        all_documents.extend(documents)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )
    splits = text_splitter.split_documents(all_documents)
    print(f"Created {len(splits)} text chunks")
    
    embeddings = get_embeddings()
    
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
    return vectorstore

def load_existing_db():
    """기존 DB 로드"""
    embeddings = get_embeddings()
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    return vectorstore

def get_vectorstore():
    """DB가 없으면 생성, 있으면 로드"""
    if not os.path.exists(PERSIST_DIRECTORY):
        print("Creating new Chroma DB...")
        return load_and_persist_pdfs()
    else:
        print("Loading existing Chroma DB...")
        return load_existing_db()

def format_docs(docs):
    """문서 포맷팅"""
    return "\n\n".join([d.page_content for d in docs])

# 메모리 설정
memory = ConversationBufferMemory(
    return_messages=True,
    output_key="output",
    input_key="input"
)

def load_memory(input_dict):
    """메모리 로드"""
    return memory.load_memory_variables({})["history"]

class PDFIngestor:
    def __init__(self):
        self.persist_directory = PERSIST_DIRECTORY
        self.pdf_directory = PDF_DIRECTORY

    def get_retriever(self):
        """벡터 스토어에서 리트리버 생성"""
        vectorstore = get_vectorstore()
        return vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
        )

class Chain:
    def __init__(self, retriever):
        # LLM 정의
        llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
        
        # 프롬프트 템플릿 정의
        prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 RFP(Request for Proposal) 작성과 분석을 돕는 전문 어시스턴트입니다.
            
            다음과 같은 RFP의 핵심 요소들을 중심으로 답변해주세요:
            1. 문제 정의 및 추진 배경
            2. 연구 목표 (최종/세부)
            3. 연구 내용 및 범위
            4. 기대 성과 및 평가 지표
            5. 특기 사항 (제약조건, 필수 요구사항 등)
            
            답변 시에는 구체적이고 실현 가능한 제안을 제시하되, 혁신성과 도전성도 균형있게 고려해주세요.
            항상 한국어로 답변하며, 가능한 명확한 근거와 예시를 포함해주세요."""),
            MessagesPlaceholder(variable_name="history"),
            ("human", "[참고 문서]\n{context}\n\n[질문/요청사항]\n{query}")
        ])
        
        # 체인 정의
        self.chain = (
            {
                'context': retriever | format_docs,
                'query': RunnablePassthrough()
            }
            | RunnablePassthrough.assign(history=load_memory)
            | prompt
            | llm
            | StrOutputParser()
        )

    def astream(self, query):
        """스트리밍 방식으로 응답 생성"""
        result = ''
        for chunk in self.chain.stream(query):
            print(chunk, end="", flush=True)
            result += chunk

        memory.save_context(
            {"input": query},
            {"output": result},
        )

def main():
    # PDF 처리기 초기화 및 리트리버 생성
    pdf_ingestor = PDFIngestor()
    retriever = pdf_ingestor.get_retriever()

    # Chain 초기화
    chain = Chain(retriever)

    print("\n=== RFP Assistant 시작 ===")
    print("'exit'를 입력하면 대화가 종료됩니다.")
    print("질문이나 RFP 작성 요청을 입력해주세요.\n")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("\n대화를 종료합니다. 감사합니다!")
            break

        print("\nAI: ", end="")
        chain.astream(user_input)
        print("\n" + "-"*50)

if __name__ == "__main__":
    main()

LangSmith 추적을 시작합니다.
[프로젝트명]
RFP-Assistant
Loading existing Chroma DB...

=== RFP Assistant 시작 ===
'exit'를 입력하면 대화가 종료됩니다.
질문이나 RFP 작성 요청을 입력해주세요.


AI: RFP(Request for Proposal) 양식에는 다음과 같은 핵심 요소들이 포함되어야 합니다. 각 요소는 명확하고 구체적으로 작성되어야 하며, 제안자들이 과제의 요구사항과 기대치를 명확히 이해할 수 있도록 해야 합니다.

1. **문제 정의 및 추진 배경**
   - 현재 해결하고자 하는 문제의 정의와 그 문제의 중요성, 시급성에 대한 설명이 필요합니다.
   - 문제 발생의 배경과 맥락을 설명하여 제안자들이 문제의 근본 원인을 이해할 수 있도록 합니다.
   - 관련된 기존 연구나 프로젝트의 결과 및 한계점도 포함될 수 있습니다.

2. **연구 목표 (최종/세부)**
   - 연구의 최종 목표와 이를 달성하기 위한 세부 목표를 명확히 기술합니다.
   - 목표는 구체적이고 측정 가능해야 하며, SMART(구체적, 측정 가능, 달성 가능, 관련성, 시간 제한) 원칙을 따르는 것이 좋습니다.

3. **연구 내용 및 범위**
   - 연구가 다루어야 할 주요 내용과 범위를 명확히 정의합니다.
   - 연구의 범위는 시간적, 공간적, 주제적 한계를 포함하여 명확히 설정해야 합니다.
   - 필요한 경우, 연구 방법론이나 접근 방식에 대한 지침을 제공할 수 있습니다.

4. **기대 성과 및 평가 지표**
   - 연구가 완료되었을 때 기대되는 성과를 구체적으로 기술합니다.
   - 성과를 평가할 수 있는 지표를 제시하여, 연구의 성공 여부를 객관적으로 판단할 수 있도록 합니다.
   - 예를 들어, 기술 개발의 경우 기술 성숙도(TRL) 수준, 경제적 효과, 사회적 영향 등을 포함할 수 있습니다.

5. **특기 사항 (제약조건, 필수 요구사항 등)**
   - 연구 수행 시 고려해야 

# LangGraph: 상태 기반 AI 워크플로우 프레임워크

## 개요
LangGraph는 LangChain 생태계의 일부로, 상태 기반의 AI 워크플로우를 구축하기 위한 프레임워크입니다. 복잡한 AI 시스템을 모듈화된 그래프 구조로 구현할 수 있게 해줍니다.

## 핵심 개념

### 1. 상태 (State)
- 워크플로우의 현재 상태를 나타내는 데이터 구조
- TypedDict를 사용하여 정의
- 노드 간 데이터 전달의 기반

### 2. 노드 (Node)
- 워크플로우의 개별 작업 단위
- 상태를 입력받아 처리하고 업데이트된 상태를 반환
- 순수 함수로 구현 권장

### 3. 엣지 (Edge)
- 노드 간의 연결을 정의
- 데이터 흐름의 방향을 결정
- 조건부 분기 가능

### 4. 체크포인팅 (Checkpointing)
- 실행 상태 추적 및 저장
- 디버깅 및 모니터링 지원
- 실행 재개 가능성 제공

## 주요 특징

### 1. 모듈성
- 각 노드는 독립적으로 개발 및 테스트 가능
- 재사용 가능한 컴포넌트로 설계
- 유지보수 용이성 향상

### 2. 유연성
- 동적 워크플로우 구성 가능
- 조건부 분기 처리
- 병렬 실행 지원

### 3. 추적성
- 실행 흐름 모니터링
- 상태 변화 추적
- 디버깅 용이성

### 4. 확장성
- 새로운 노드 쉽게 추가
- 기존 워크플로우 수정 용이
- 복잡한 시스템 구현 가능

## 일반적인 사용 사례

### 1. 대화형 시스템
- 멀티턴 대화 관리
- 컨텍스트 유지
- 대화 흐름 제어

### 2. 복잡한 추론 시스템
- Plan-and-Execute 패턴
- ReAct 패턴
- 멀티스텝 추론

### 3. 워크플로우 자동화
- 문서 처리 파이프라인
- 데이터 변환 및 강화
- 품질 관리 프로세스

## 모범 사례

### 1. 상태 설계
- 명확한 타입 정의
- 최소한의 필요 데이터만 포함
- 불변성 고려

### 2. 노드 구현
- 단일 책임 원칙 준수
- 부작용 최소화
- 에러 처리 포함

### 3. 워크플로우 구성
- 논리적 흐름 설계
- 적절한 에러 처리 경로
- 확장성 고려

### 4. 테스트 및 디버깅
- 단위 테스트 작성
- 통합 테스트 구현
- 로깅 전략 수립

## 장점

1. **구조화된 접근**
   - 복잡한 로직을 관리 가능한 단위로 분할
   - 명확한 데이터 흐름
   - 코드 구조 개선

2. **유지보수성**
   - 모듈화된 구조
   - 독립적인 컴포넌트
   - 테스트 용이성

3. **확장성**
   - 새로운 기능 추가 용이
   - 기존 로직 수정 간편
   - 재사용 가능한 컴포넌트

4. **디버깅**
   - 상태 변화 추적
   - 실행 흐름 모니터링
   - 오류 지점 식별 용이

## 결론

LangGraph는 복잡한 AI 시스템을 구조화된 방식으로 구현할 수 있게 해주는 강력한 도구입니다. 상태 기반 접근방식과 모듈화된 구조는 확장 가능하고 유지보수하기 쉬운 시스템을 구축하는데 매우 효과적입니다.

In [ ]:
# !pip install graphviz > 그래프를 보기 위한 라이브러리 (필요시에만 진행)

In [ ]:
import os
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.models import get_model_name, LLMs
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
# LangGraph 관련 임포트
from langgraph.graph import StateGraph, START, END  # 그래프 구조 정의를 위한 기본 클래스
from langgraph.checkpoint.memory import MemorySaver  # 실행 상태 추적을 위한 체크포인터
from langchain_core.runnables import RunnableConfig  # 실행 설정을 위한 클래스
from uuid import uuid4
from typing import Annotated, TypedDict, List
from typing_extensions import TypedDict
import glob
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# API 키 로드
load_dotenv()

# LangSmith 설정
logging.langsmith("RFP-Assistant")

# 모델명 정의
MODEL_NAME = get_model_name(LLMs.GPT4o)

# 캐시 디렉토리 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

# DB 저장 경로 설정
PERSIST_DIRECTORY = os.path.join('C:/Users/user/Documents/langchain-kr/어진_RFP', 'db')
PDF_DIRECTORY = 'C:/Users/user/Documents/langchain-kr/어진_RFP'

def get_embeddings():
    """HuggingFace 임베딩 모델 설정"""
    model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {"normalize_embeddings": True}
    
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder="./cache/"
    )

def load_all_pdfs() -> List[str]:
    """폴더 내의 모든 PDF 파일 경로를 반환"""
    pdf_pattern = os.path.join(PDF_DIRECTORY, '*.pdf')
    return glob.glob(pdf_pattern)

def load_and_persist_pdfs():
    """모든 PDF를 로드하고 Chroma DB에 저장"""
    pdf_files = load_all_pdfs()
    print(f"Found {len(pdf_files)} PDF files")
    
    all_documents = []
    for pdf_path in pdf_files:
        print(f"Loading {os.path.basename(pdf_path)}...")
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        all_documents.extend(documents)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )
    splits = text_splitter.split_documents(all_documents)
    print(f"Created {len(splits)} text chunks")
    
    embeddings = get_embeddings()
    
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
    return vectorstore

def load_existing_db():
    """기존 DB 로드"""
    embeddings = get_embeddings()
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    return vectorstore

def get_vectorstore():
    """DB가 없으면 생성, 있으면 로드"""
    if not os.path.exists(PERSIST_DIRECTORY):
        print("Creating new Chroma DB...")
        return load_and_persist_pdfs()
    else:
        print("Loading existing Chroma DB...")
        return load_existing_db()

def format_docs(docs):
    """문서 포맷팅"""
    return "\n\n".join([d.page_content for d in docs])

# LangGraph의 상태를 정의하는 클래스
# TypedDict를 사용하여 상태의 구조를 명시적으로 정의
class RFPState(TypedDict):
    """RFP Assistant의 상태를 정의하는 클래스
    
    Attributes:
        query: 사용자의 입력 질문
        context: 검색된 문서 컨텍스트
        chat_history: 대화 히스토리 리스트
        response: AI의 최종 응답
    """
    query: Annotated[str, "사용자 입력"]
    context: Annotated[str, "검색된 문서 컨텍스트"]
    chat_history: Annotated[list, "대화 히스토리"]
    response: Annotated[str, "최종 응답"]

# LangGraph 노드 함수 정의
def retrieve_context(state: RFPState):
    """문서 검색 노드
    
    Args:
        state: 현재 워크플로우 상태
        
    Returns:
        dict: 업데이트된 상태 (context 키 포함)
    """
    query = state["query"]
    retriever = get_vectorstore().as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}
    )
    docs = retriever.get_relevant_documents(query)
    context = format_docs(docs)
    return {"context": context}

def generate_response(state: RFPState):
    """응답 생성 노드
    
    Args:
        state: 현재 워크플로우 상태
        
    Returns:
        dict: 업데이트된 상태 (response 키 포함)
    """
    llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
    prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 RFP(Request for Proposal) 작성과 분석을 돕는 전문 어시스턴트입니다.
        
        다음과 같은 RFP의 핵심 요소들을 중심으로 답변해주세요:
        1. 문제 정의 및 추진 배경
        2. 연구 목표 (최종/세부)
        3. 연구 내용 및 범위
        4. 기대 성과 및 평가 지표
        5. 특기 사항 (제약조건, 필수 요구사항 등)
        
        답변 시에는 구체적이고 실현 가능한 제안을 제시하되, 혁신성과 도전성도 균형있게 고려해주세요."""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "[참고 문서]\n{context}\n\n[질문/요청사항]\n{query}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({
        "context": state["context"],
        "query": state["query"],
        "chat_history": state["chat_history"]
    })
    
    return {"response": response}

def update_history(state: RFPState):
    """대화 히스토리 업데이트 노드
    
    Args:
        state: 현재 워크플로우 상태
        
    Returns:
        dict: 업데이트된 상태 (chat_history 키 포함)
    """
    current_history = state["chat_history"]
    current_history.extend([
        ("human", state["query"]),
        ("assistant", state["response"])
    ])
    return {"chat_history": current_history}

def create_rfp_graph():
    """RFP Assistant 그래프 생성
    
    LangGraph를 사용하여 워크플로우를 정의하고 컴파일합니다.
    
    Returns:
        compiled_graph: 컴파일된 LangGraph 워크플로우
    """
    # StateGraph 인스턴스 생성 (상태 타입 지정)
    workflow = StateGraph(RFPState)
    
    # 노드 추가 (각 작업 단계 정의)
    workflow.add_node("retrieve", retrieve_context)  # 문서 검색 노드
    workflow.add_node("generate", generate_response)  # 응답 생성 노드
    workflow.add_node("update_history", update_history)  # 히스토리 업데이트 노드
    
    # 엣지 추가 (노드 간 연결 정의)
    workflow.add_edge(START, "retrieve")  # 시작 → 문서 검색
    workflow.add_edge("retrieve", "generate")  # 문서 검색 → 응답 생성
    workflow.add_edge("generate", "update_history")  # 응답 생성 → 히스토리 업데이트
    workflow.add_edge("update_history", END)  # 히스토리 업데이트 → 종료
    
    # 그래프 컴파일 (체크포인터 설정)
    return workflow.compile(checkpointer=MemorySaver())


def visualize_workflow():
    """
    LangGraph 워크플로우를 시각화하는 함수
    
    이 함수는 다음을 수행합니다:
    1. 워크플로우 그래프 생성
    2. Graphviz를 사용한 시각화
    3. 상세 정보 표시 (xray 모드)
    """
    try:
        from graphviz import Digraph
        
        # 워크플로우 그래프 생성
        workflow = create_rfp_graph()
        
        # 그래프 객체 생성
        dot = Digraph(comment='RFP Assistant Workflow')
        dot.attr(rankdir='LR')
        
        # 노드 스타일 설정
        dot.attr('node', shape='box', style='rounded', fontname='Arial')
        
        # 노드 색상 설정
        node_colors = {
            'START': 'lightgreen',
            'END': 'lightpink',
            'retrieve': 'lightblue',
            'generate': 'lightyellow',
            'update_history': 'lightgrey'
        }
        
        # 노드 추가 (색상과 함께)
        for node, color in node_colors.items():
            shape = 'circle' if node in ['START', 'END'] else 'box'
            dot.node(node, node, shape=shape, style='filled', fillcolor=color)
        
        # 엣지 추가
        dot.edge('START', 'retrieve')
        dot.edge('retrieve', 'generate')
        dot.edge('generate', 'update_history')
        dot.edge('update_history', 'END')
        
        # 그래프 저장
        try:
            dot.render("rfp_workflow", format="png", cleanup=True)
            print("워크플로우 그래프가 'rfp_workflow.png'로 저장되었습니다.")
            return dot
        except Exception as e:
            print(f"그래프 저장 중 오류 발생: {str(e)}")
            return None
            
    except ImportError:
        print("\nGraphviz 설치가 필요합니다:")
        print("1. pip install graphviz")
        print("2. 시스템에 Graphviz 설치:")
        print("   - Windows: https://graphviz.org/download/")
        print("   - Ubuntu: sudo apt-get install graphviz")
        print("   - Mac: brew install graphviz")
        return None
    except Exception as e:
        print(f"\n시각화 중 오류 발생: {str(e)}")
        return None

# 시각화 실행 (주피터 노트북 셀에서 실행)
if __name__ == "__main__":
    # 워크플로우 시각화
    graph = visualize_workflow()
    if graph:
        # 그래프를 PNG 파일로 저장
        graph.render("rfp_workflow", format="png", cleanup=True)
        print("워크플로우 그래프가 'rfp_workflow.png'로 저장되었습니다.")

def main():
    # 그래프 생성
    app = create_rfp_graph()
    
    # 초기 상태 설정
    initial_state = {
        "chat_history": [],
    }
    
    print("\n=== RFP Assistant 시작 ===")
    print("'exit'를 입력하면 대화가 종료됩니다.")
    print("질문이나 RFP 작성 요청을 입력해주세요.\n")
    
    # LangGraph 체크포인터 설정
    # - thread_id: 대화 세션 식별
    # - checkpoint_ns: 체크포인트 네임스페이스
    # - checkpoint_id: 개별 체크포인트 식별
    config = RunnableConfig(
        configurable={
            "thread_id": str(uuid4()),
            "checkpoint_ns": "rfp_assistant",
            "checkpoint_id": str(uuid4()),
        }
    )
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("\n대화를 종료합니다. 감사합니다!")
            break
        
        # 현재 상태 업데이트
        current_state = {
            **initial_state,
            "query": user_input,
        }
        
        # 그래프 실행 (상태와 설정 전달)
        print("\nAI: ", end="")
        result = app.invoke(current_state, config=config)
        print(result["response"])
        
        # 대화 히스토리 업데이트
        initial_state["chat_history"] = result["chat_history"]
        print("\n" + "-"*50)

if __name__ == "__main__":
    main()